In [1]:
import pandas as pd
from kiwipiepy import Kiwi
import re
import copy


# 필요한 객체 및 변수 정의
kiwi = Kiwi(typos='basic') #typos = basic -> 토큰화(형태소 분석)시에 기본적인 오탈자 교정해줌

# 분석 대상 키워드
query = '교회' 
df = pd.read_csv(f'./Raw_Content/기독교_본문/{query}_content.csv') 
# 본문(content) 스크래핑 결과가 Nan인 값을 제거
df = df.dropna(subset=['body'],ignore_index=True)

In [2]:
# 각 종교별 키워드를 불용어 처리에 사용 - 종교적 맥락을 너무 저해한다고 판단 주석처리 
# Christianity_stop = ['기독교','교회','하나님','개신교','목사','예수']
# Catholicism_stop = ['천주교','수녀님','성당','하느님','신부님','교황','추기경']
# Buddhism_stop = ['불교','부처님','절','스님','승려','보살']
news_stop = ['해당 기사', '제공', '이메일', '카카오톡', '제보', '노컷뉴스', '무단', '전재','배포']
add_stop = ['상황', '곳', '뒤', '후', '앞', '이번','종교',]

In [ ]:
#종교별 데이터수 균형을 맞추기 위함함
#df = df.head(1000)

In [4]:
from kiwipiepy.utils import Stopwords # kiwi에서 제공하는 Stopword list
stopwords = Stopwords()

# 기본 불용어 + 사용자 불용어 통합
default_stopwords = stopwords.stopwords
stopwords = set(default_stopwords)  # 기본 불용어를 set으로 변환
custom_stopwords = set(news_stop + add_stop) #+ Christianity_stop + Catholicism_stop + Buddhism_stop )

stopwords.update((word, 'NNG') for word in custom_stopwords)  # 사용자 불용어 추가 (보통명사)
stopwords.update((word, 'NNP') for word in custom_stopwords)  # 사용자 불용어 추가 (고유명사)

In [5]:
p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')  # 한글만 필터링 

def preprocess_korean(text, analyzer=kiwi, stopwords=stopwords, return_list=False):
    my_text = str(text).replace('\n', ' ')  # 개행 제거
    my_text = analyzer.space(my_text)  # 띄어쓰기 교정

    
    sents = analyzer.split_into_sents(my_text) #문장 토큰화
    all_result = []

    for sent in sents:
        token_result = analyzer.tokenize(sent.text)#단어 토큰화
        tmp = []  # 
        for token in token_result:
            # 명사(NNG, NNP, 동사, 형용사)만 추출 & 불용어 제거 -> 명사 대명사만 필터링
            if (token.form, token.tag) not in stopwords and token.tag in ['NNG', 'NNP']:
                tmp.append(token.form)

        token_result = ' '.join(tmp)
        token_result = p.sub(' ', token_result)  # 한글 이외 문자 제거
        all_result.append(token_result)

    return all_result if return_list else ' '.join(all_result)  # 리스트로 반환 옵션 추가


In [ ]:
df['pre_body'] = df['body'].apply(lambda x: preprocess_korean(x)) # 전처리기 적용
df.to_csv(f'../modeling/Preprocessed_Content/preprocessed_{query}.csv', index=False)  # 전처리된 결과를 담은 csv 저장